# Imports

In [ ]:
%pip install pandas==2.0.3
%pip install tqdm==4.66.1
%pip install pm4py==2.7.8.2
%pip install pygraphviz==1.11

In [1]:
import json
import pandas as pd
import pm4py
import networkx as nx
from networkx.algorithms import isomorphism
from enum import Enum, auto
from IPython.display import display, HTML
from typing import Tuple, Dict

from pm4py.objects.bpmn.obj import BPMN

# Process Mining

In [17]:
import os
import json
import pandas as pd
import pm4py
from pm4py.objects.bpmn.obj import BPMN
from IPython.display import display, HTML

def print_bpmn(bpmn_graph):
    # Print the structure of the BPMN graph
    for node in bpmn_graph.get_nodes():
        print(f"Node: {node.get_name()}, Type: {type(node).__name__}")
    for flow in bpmn_graph.get_flows():
        print(f"Flow: {flow.get_source().get_name()} -> {flow.get_target().get_name()}")

class ProcessingMode(Enum):
    MODEL = auto()

def discover_bpmn_graph(log_data):        
    """Extract Petri net graph from log data."""
    logs_df = pd.DataFrame(log_data)   
    logs_df['timestamp'] = pd.to_datetime(logs_df['timestamp'])
    logs_df['logging_statement_id'] = logs_df['logging_statement_id'].astype(str)
    logs_df['case_id'] = '01'
    
    params = {
        'dependency_threshold': 0.45,
        'and_threshold': 0.65,
        'loop_two_threshold': 0.5,
        'activity_key': 'logging_statement_id',
        'case_id_key': 'case_id',
        'timestamp_key': 'timestamp'
    }
    net, im, fm = pm4py.discover_petri_net_heuristics(logs_df, **params)
    
    bpmn_graph = pm4py.convert_to_bpmn(net, im, fm)
    
    # Get all unique logging_statement_ids from DataFrame
    unique_logging_statement_ids = set(logs_df['logging_statement_id'].unique())

    # Check if all unique_logging_statement_ids are in the tasks of the BPMN graph
    bpmn_tasks = [node for node in bpmn_graph.get_nodes() if isinstance(node, BPMN.Task)]
    task_labels = set(task.get_name() for task in bpmn_tasks)

    missing_ids = unique_logging_statement_ids - task_labels

    if missing_ids:
        missing_ids_str = '<br>'.join(missing_ids)
        display(HTML(f"<span style='color: red;'>Missing logging_statement_ids in BPMN graph:<br>{missing_ids_str}</span>"))
    else:
        display(HTML(f"<span style='color: green;'>All logging_statement_ids accounted for</span>"))
            
    bpmn_graph = pm4py.convert_to_bpmn(net, im, fm)
    
    return bpmn_graph

def load_logs(base_dir, mode):
    """Load logs from a file."""
    with open(f'{base_dir}/processed_logs_mode_{mode.name}.json') as f:
        return json.load(f)

def get_unique_services_and_subprocesses(logs, logs_B=None):
    """
    Get unique services and subprocesses.
    If logs_B is provided, it returns the unique items between the two sets of logs.
    If logs_B is None, it returns the unique items from logs.
    """
    if logs_B:
        unique_services = set(logs.keys()).union(set(logs_B.keys()))
    else:
        unique_services = set(logs.keys())

    unique_subprocesses = {}
    for service in unique_services:
        subprocesses_A = set(logs.get(service, {}).keys())
        subprocesses_B = set(logs_B.get(service, {}).keys()) if logs_B else set()
        unique_subprocesses[service] = subprocesses_A.union(subprocesses_B)

    return unique_services, unique_subprocesses

def create_empty_bpmn():
    """Create an empty BPMN graph with a single start event."""
    empty_bpmn = BPMN(name='Empty BPMN')
    return empty_bpmn

def process_graphs_and_save(logs, all_process_graphs, unique_services, unique_subprocesses):
    """Generate process graphs and save them."""
    for mode in ProcessingMode:
        all_process_graphs.setdefault(mode.name, {})
        for service in unique_services[mode]:
            for subprocess in unique_subprocesses[mode].get(service, {}):
                target_logs = logs[mode.name].get(service, {}).get(subprocess, {}).get('logs', [])
                
                 # Check if target_logs is empty
                if not target_logs:
                    print(f"No logs found for {service} -> {subprocess} in {mode.name}. Creating an empty BPMN graph.")
                    bpmn_graph = create_empty_bpmn()
                else:
                    bpmn_graph = discover_bpmn_graph(target_logs)
                    
                # print_bpmn(bpmn_graph)    
                
                # Save net along with its initial and final markings
                all_process_graphs[mode.name].setdefault(service, {})[subprocess] = {
                    'bpmn_graph': bpmn_graph,
                }

def process_logs(logs, process_graph):
    unique_services = {}
    unique_subprocesses = {}
    for mode in ProcessingMode:
        unique_services[mode], unique_subprocesses[mode] = get_unique_services_and_subprocesses(logs[mode.name])

    process_graphs_and_save(logs, process_graph, unique_services, unique_subprocesses)
    display(HTML(f"<b><span style='color: blue;'>&emsp;&emsp; --> DONE</span></b>"))


process_graph = {}
logs = {mode.name: load_logs('../Data/Reproduced Experiment/Preprocessed/Logs/Part A/', mode) for mode in ProcessingMode}    
process_logs(logs, process_graph)  

# Gateway Identification Using Path-Based Signiture

In [18]:
def extract_gateways(bpmn_graph):
    """
    This function extracts gateways from a BPMN model and identifies them based on their path-based signatures.
    It maps each gateway to the unique paths leading from it, considering only Events and Tasks as path terminators.
    The paths are stored as tuples of node names, ensuring a consistent identification system.
    This approach allows for the identification of gateways even when only partial information about unique IDs is available,
    as it relies on the structural signature of the paths rather than solely on the IDs.
    The output is a sorted dictionary where each key represents a unique path structure (as a JSON string),
    and the value is a list of gateways sharing that structure. This forms the basis for consistent node identification across different BPMN models.
    """
    # Initialize dictionaries for storing paths by gateway and gateway structures.
    paths_by_gateway = {}
    structure_to_gateways = {}

    # Define a depth-first search (DFS) function to traverse the graph.
    def dfs(current_node, current_path, visited_gateways):
        # Check if current node is an Event or Task, marking the end of a path.
        if isinstance(current_node, (BPMN.Event, BPMN.Task)):
            current_path_tuple = tuple(current_path)
            paths_by_gateway[start_gateway_id].append(current_path_tuple)
            return

        # Iterate over outgoing arcs from the current node.
        for arc in current_node.get_out_arcs():
            next_node = arc.get_target()
            # Process next node if it is a Gateway, Event, or Task and not already visited.
            if isinstance(next_node, (BPMN.Gateway, BPMN.Event, BPMN.Task)) and next_node not in visited_gateways:
                # Get node name, using type name for gateways and node name otherwise.
                node_name = type(next_node).__name__ if isinstance(next_node, BPMN.Gateway) else next_node.get_name()
                current_path.append(node_name)
                # Mark gateway as visited.
                if isinstance(next_node, BPMN.Gateway):
                    visited_gateways.add(next_node)
                # Recursively call DFS on the next node.
                dfs(next_node, current_path, visited_gateways)
                # Backtrack by removing the last node from the path and unmarking the gateway.
                current_path.pop()
                if isinstance(next_node, BPMN.Gateway):
                    visited_gateways.remove(next_node)

    # Start DFS for each gateway node in the BPMN model.
    for node in bpmn_graph.get_nodes():
        if isinstance(node, BPMN.Gateway):
            start_gateway_id = node.id
            paths_by_gateway[start_gateway_id] = []
            dfs(node, [f"{type(node).__name__}_Source"], set())

    # Group and sort gateways based on their path structures.
    for gateway, paths in paths_by_gateway.items():
        # Sort each path
        sorted_paths = sorted(paths)
        # Convert each sorted path to a string
        paths_strings = ['; '.join(path) for path in sorted_paths]
        # Convert sorted list of path strings to a JSON string
        paths_key = json.dumps(paths_strings)
        structure_to_gateways.setdefault(paths_key, []).append(gateway)   
    
    sorted_structure_to_gateways = {k: structure_to_gateways[k] for k in sorted(structure_to_gateways)}

    return sorted_structure_to_gateways
            
def handle_gateways_for_model(gateway_paths):
    # Mapping for new names of gateways.
    name_mappings = {}
    
    # Assign new names to gateways based on their path structure.
    for path_structure, gateways in gateway_paths.items():
        # Generate a hash value for the path structure string.
        path_hash = hash(path_structure)

        # Ensure the hash value is non-negative and convert it to a string.
        path_hash_str = str(abs(path_hash))

        for gateway in gateways:
            new_name = f"Gateway_{path_hash_str}"
            name_mappings[gateway] = new_name

    return name_mappings

def rebuild_bpmn_graph(bpmn_graph, gateway_mapping):
    # Create a new BPMN model with the same name.
    new_bpmn_graph = BPMN(name=bpmn_graph.get_name())
    node_mapping = {}

    # Add nodes to the new model, applying the gateway mapping.
    sorted_nodes = sorted(bpmn_graph.get_nodes(), key=lambda node: node.get_name())
    for node in sorted_nodes:
        new_id = gateway_mapping.get(node.get_id(), node.get_name())
        new_node = type(node)(id=new_id, name=new_id, process=node.get_process())
        new_bpmn_graph.add_node(new_node)
        node_mapping[node] = new_node

    # Add flows to the new model, mapping source and target nodes.
    sorted_flows = sorted(bpmn_graph.get_flows(), key=lambda flow: (flow.get_source().get_name(), flow.get_target().get_name()))
    for flow in sorted_flows:
        new_source = node_mapping[flow.get_source()]
        new_target = node_mapping[flow.get_target()]
        new_flow = type(flow)(source=new_source, target=new_target, id=flow.get_id(), name=flow.get_name(), process=flow.get_process())
        new_bpmn_graph.add_flow(new_flow)
        
    return new_bpmn_graph

# Main processing function
def process_bpmn_graph(bpmn_graph):
     # Extract and sort gateways to establish unique IDs.
    gateway_paths = extract_gateways(bpmn_graph)

    # Generate new names for gateways based on their path structures.
    name_mappings = handle_gateways_for_model(gateway_paths)

    # Rebuild the BPMN model with new gateway names.
    new_bpmn_graph = rebuild_bpmn_graph(bpmn_graph, name_mappings)
    
    return new_bpmn_graph


for mode in ProcessingMode:
    for service in process_graph[mode.name]:
        for subprocess in process_graph[mode.name][service]:
            bpmn_graph = process_graph[mode.name][service][subprocess]['bpmn_graph']
            process_graph[mode.name][service][subprocess]['bpmn_graph'] = process_bpmn_graph(bpmn_graph)
print("Done")            

Done


# A*

In [19]:
import heapq

def heuristic(graph: nx.DiGraph, source: str, target: str) -> float:
    return 0 # We dont have a grid

def a_star_search(graph: nx.DiGraph, start: str, goal: str, logs: list, require_loop=False):
    """
    Modified A* Search Algorithm to find loops.
    
    :param graph: NetworkX directed graph
    :param start: Start node ID
    :param goal: Target node ID
    :param logs: List of logs with logging_statement_id
    :return: Shortest path from start to goal, including loops if required
    """
    log_names = [log['logging_statement_id'] for log in logs]  # Extract log names from logs
    open_list = [(0, start, [])]  # Initialize open list with start node
    g_score = {start: 0}  # Initialize g_score for start node as 0
    
    while open_list:  # Loop until open list is empty
        f_score, current, path = heapq.heappop(open_list)  # Pop node with lowest f_score
        
        # If current node is the goal, and loop requirement is met, return path
        if current == goal:
            if not require_loop or (require_loop and len(path) > 1):
                return path + [current]
        
        for neighbor, edge_data in graph[current].items():  # Loop through neighbors
            # Skip nodes in logs except the goal
            if neighbor in log_names and neighbor != goal:
                continue
            
            # Calculate tentative g_score for neighbor
            tentative_g_score = g_score[current] + edge_data.get('weight', 1)
            
            # Update g_score if new path is better or neighbor is not in open list
            if tentative_g_score < g_score.get(neighbor, float('inf')) or neighbor not in [i[1] for i in open_list]:
                g_score[neighbor] = tentative_g_score
                f_score = tentative_g_score + heuristic(graph, neighbor, goal)
                heapq.heappush(open_list, (f_score, neighbor, path + [current]))
                
    return None  # Return None if path is not found



In [20]:
from collections import defaultdict

def initialize_graph_attributes(graph: nx.DiGraph):
    for node in graph.nodes():
        # Initialize node attributes using the graph object
        if f'count' not in graph.nodes[node]:
            graph.nodes[node][f'count'] = 0

    for u, v in graph.edges():
        # Initialize edge attributes using the graph object
        if f'count' not in graph[u][v]:
            graph[u][v][f'count'] = 0

def update_path_attributes(graph, path):
    """
    Update attributes of edges in the given path.

    :param graph: NetworkX graph
    :param path: List of nodes forming a path
    """
    for i in range(len(path) - 1):
        edge = (path[i], path[i + 1])
        
        # Initialize count for the edge if not present
        if f'count' not in graph.edges[edge]:
            graph.edges[edge][f'count'] = 0
            
        # Increment the count for the edge
        graph.edges[edge][f'count'] += 1
    
    # Update node attributes
    for i, node in enumerate(path[1:-1]):  # Exclude the start and end nodes
        # Initialize count for the node if not present
        if f'count' not in graph.nodes[node]:
            graph.nodes[node][f'count'] = 0
            
        # Increment the count for the node
        graph.nodes[node][f'count'] += 1

            
def update_graph_from_logs(graph: nx.DiGraph, logs: list):
    """
    Updates graph attributes based on logs
    """
    initialize_graph_attributes(graph)
    
    # Step 1: Count occurrences of each logging_statement_id in logs
    log_count = defaultdict(int)
    for log in logs:
        log_count[log['logging_statement_id']] += 1
        
    for i in range(len(logs) - 1):
        start_node = logs[i]['logging_statement_id']
        end_node = logs[i + 1]['logging_statement_id']
        
        require_loop = start_node == end_node 
        
        if start_node not in graph or end_node not in graph:
            missing_type = []
            if start_node not in graph:
                missing_type.append(f"Start node: {start_node}")
            if end_node not in graph:
                missing_type.append(f"End node: {end_node}")
            
            continue
        
        # A* search to find path between start_node and end_node
        path = a_star_search(graph, start_node, end_node, logs, require_loop)
        if path:
            update_path_attributes(graph, path)                   
        # else:
        #     # Print a warning if the path is not found
        #     display(HTML(f"<b><span style='color: orange;'>&emsp;&emsp; --> Path not found between {start_node} and {end_node} in Model {model} - Require Loop {require_loop}.</span></b>"))
            
            # display(HTML(f"<b><span style='color: orange;'>&emsp;&emsp; Attempting to add bridge...</span></b>"))
             # Add a dashed edge with isBridge=True attribute
#             graph.add_edge(start_node, end_node, style='dashed', isBridge=True)
            # plot_graph_with_missing_path(graph, start_node, end_node, title=f"Missing path {start_node} to {end_node} in Model {model}")
            
#             new_path = a_star_search(graph, start_node, end_node)
#             if new_path:
#                 # Update the path attributes using the new path
#                 update_path_attributes(graph, new_path, model)
#             else:
#                 display(HTML(f"<b><span style='color: red;'>&emsp;&emsp; --> Path not found between {start_node} and {end_node} in Model {model}</span></b>"))

    # Set each node's count that corresponds to a log to that log count
    for node, count in log_count.items():
        if node in graph:
            graph.nodes[node][f'count'] = count

    # # Step 2: Validate counts
    # for node, attr in graph.nodes(data=True):
    #     if attr.get('type') == 'transition':
    #         if f'count_{model}' in attr:
    #             if attr[f'count_{model}'] != log_count.get(node, 0):
    #                 display(HTML(f"<b><span style='color: red;'>&emsp;&emsp; --> Mismatch in counts for node {node} in Model {model}. Count in graph: {attr[f'count_{model}']}, Count in logs: {log_count.get(node, 0)}</span></b>"))

# Graph Construction

In [21]:
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import to_agraph

def plot_graph(graph, title):

    
    A = to_agraph(graph)
    
    for node in A.nodes():
        node_name = node.name
        node_type = graph.nodes[node_name].get('type', None)
        
        if node_type == 'StartEvent':
            node.attr['shape'] = 'circle'
            node.attr['label'] = ''
            node.attr['width'] = 0.6
            node.attr['height'] = 0.6
            node.attr['fillcolor'] = 'green'
            node.attr['style'] = 'filled'    
            
        elif node_type == 'NormalEndEvent':
            node.attr['shape'] = 'circle'
            node.attr['label'] = ''
            node.attr['width'] = 0.6
            node.attr['height'] = 0.6
            node.attr['fillcolor'] = 'orange'
            node.attr['style'] = 'filled'      
            
        elif node_type == 'Task':
            node.attr['shape'] = 'rectangle'
          
        elif node_type == 'ExclusiveGateway':
            node.attr['shape'] = 'diamond'
            node.attr['label'] = 'X'
            node.attr['width'] = 0.6
            node.attr['height'] = 0.6
            node.attr['style'] = 'filled'

        elif node_type == 'ParallelGateway':
            node.attr['shape'] = 'diamond'
            node.attr['label'] = '+'
            node.attr['width'] = 0.6
            node.attr['height'] = 0.6
            node.attr['style'] = 'filled'  
        
        else :
            print(f"Could not handle node of type: {node_type}")
            
            
    A.layout(prog='dot')
    
    plt.figure(figsize=(20, 10))
    plt.axis('off')
    plt.title(title)
    A.draw(f"Graphs/Single/{title}.png")
    img = plt.imread(f"Graphs/Single/{title}.png")
    plt.imshow(img)
    plt.show()

In [22]:
import os
from networkx.drawing.nx_agraph import to_agraph, write_dot
from pm4py.objects.petri_net.obj import PetriNet
from pm4py.objects.petri_net.utils.networkx_graph import create_networkx_directed_graph
from pm4py.objects.petri_net.utils import petri_utils

def print_networkx_graph_structure(G):
    """
    Print the structure of a NetworkX graph.
    """
    print("Nodes in the Graph:")
    for node, attrs in G.nodes(data=True):
        print(f"Node: {node}, Attributes: {attrs}")

    print("\nEdges in the Graph:")
    for edge in G.edges():
        print(f"Edge from {edge[0]} to {edge[1]}")
        
def bpmn_to_networkx(bpmn_model):
    """
    Convert a BPMN model to a NetworkX graph.
    """
    G = nx.DiGraph()

    # Add nodes to the NetworkX graph using names
    for node in bpmn_model.get_nodes():
        G.add_node(node.name, label=node.name, type=type(node).__name__)

    # Add edges to the NetworkX graph using names
    for flow in bpmn_model.get_flows():
        source_name = flow.source.name if hasattr(flow.source, 'name') else None
        target_name = flow.target.name if hasattr(flow.target, 'name') else None
        if source_name and target_name:
            G.add_edge(source_name, target_name)

    return G

def annotate_single_graph(G):
    """
    Annotate the single graph nodes and edges based on their attributes and log data.
    """
    # Annotate nodes
    for node in G.nodes():
        G.nodes[node].setdefault('count', 0)
        G.nodes[node].setdefault('in_graph', True)

    # Annotate edges
    for u, v, attributes in G.edges(data=True):
        G.edges[(u, v)].setdefault('count', 0)
        G.edges[(u, v)].setdefault('in_graph', True)

        
def save_and_plot_single_model(process_graph, raw_log_data):
    """
    Saves and plots all models based on raw log data for a single model.
    """
    json_data = {}
    
    service_names = []
    
    for mode_name, services in process_graph.items():
        
        json_data[mode_name] = {
        "viewType": "Single", 
        "graphType": "BPMN", 
        "data": {} 
        }
        
        for service_name, subprocesses in services.items():
            
            service_names.append(service_name)
            
            json_data[mode_name]["data"][service_name] = {}
            
            for subprocess_name, model_data in subprocesses.items():
                bpmn_graph = model_data.get('bpmn_graph', None)
                if bpmn_graph is None:
                    continue

                G = bpmn_to_networkx(bpmn_graph)
                log_data = raw_log_data.get(mode_name, {}).get(service_name, {}).get(subprocess_name, {})   
                annotate_single_graph(G)
                update_graph_from_logs(G, log_data.get('logs', []))
                title = f"{service_name} - {subprocess_name}"
                # print_networkx_graph_structure(G)
                # plot_graph(G, title)

                temp_dot_file = f"temp_graph.dot"
                write_dot(G, temp_dot_file)
                with open(temp_dot_file, 'r') as file:
                    graph_content = file.read()
                os.remove(temp_dot_file)  
                
                json_data[mode_name]["data"][service_name][subprocess_name] = {
                    "graphData": graph_content,
                    "logData": log_data
                }
                
        
        # Add Blackbox Services
        service_names.append("web-app")
        service_names.append("auth-service")

        json_data[mode_name]["services"] = service_names
            
        save_json(json_data[mode_name], f"../Data/Reproduced Experiment/Diagnosis/Models/Part A/model_partA.json")
        
def save_json(data, file_path):
    """Save a Python dictionary to a JSON file."""
    print(f"Saving {file_path}")
    
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)
        
        
# Call the function to save and plot all models
save_and_plot_single_model(process_graph, logs)
print("DONE")

Saving ../Data/Reproduced Experiment/Diagnosis/Models/Part A/model_partA.json
DONE
